## how to handle concat dataframes with different indices
1. `pd.concat`, `pd.merge` , `df.join` all works, with method specified as `inner | outer | left | ...`
2. but we should be especially careful about the duplicated indexes, which would lead to the error of `InvalidIndexError: Reindexing only valid with uniquely valued Index objects`

In [1]:
import pymongo
from pprint import pprint
from datetime import datetime

In [2]:
client = pymongo.MongoClient()
db = client['sjm_dnf_prices']
coll_price = db['price']
len(list(coll_price.find({})))

1891

In [15]:
ONLY_AVG_PRICE = True

In [18]:
def drop_duplicates_by_index(df, printAtEnd=True):
    # we should be especially careful about the duplicated indexes, which would lead to the error of `InvalidIndexError: Reindexing only valid with uniquely valued Index objects`
    duplicates = df[df.index.duplicated()]
    if len(duplicates):
        if not printAtEnd:
            for _, row in duplicates.iterrows():
                print(f'duplicated: {dict(date=_, **row)}')
    return df[~df.index.duplicated()]

def getCurTime():
    return datetime.now().strftime('%m-%dT%H-%M')

def getItemName(item: dict) -> str:
    try:
        return f'{item["category"]["L1_name"]}-{item["category"]["L2_name"]}-{item["name"]}'
    except Exception as e:
        print(item)
        raise e
        
def getIndex(item: dict):
    return item['data']['xAxis'][0]['data']

def item2df(item: dict, printAtEnd=True) -> pd.DataFrame:
    itemName = getItemName(item)
    if not printAtEnd:
        print(f'handling: {itemName}')
    
    data = item['data']
    df_dict = {}
    for series in [*data['xAxis'], *data['series']]:
        name = series['name']
        if ONLY_AVG_PRICE and name in ['日期', '平均价']:
            values = series['data']
            if name == '平均价':
                name = itemName
            df_dict[name] = values
    df = pd.DataFrame(df_dict)

    df.set_index('日期', inplace=True)

    df = drop_duplicates_by_index(df)
    
    df.name = itemName
    
    if printAtEnd:
        pass
#         print(f'handled: {itemName}, duplicatedCount: {len(duplicates)}')
    return df

def joinDFs(dataframes, keys=None):
    keys = None if ONLY_AVG_PRICE else (keys or [i.name for i in dataframes])
    df = pd.concat(dataframes, axis=1, keys=keys)
    return df.sort_index()

N = 0
itemsList = list(coll_price.find({}).limit(N))

In [19]:
# df774 = item2df(itemsList[774])
# df775 = item2df(itemsList[775])

dfs = joinDFs([item2df(item) for item in itemsList])
dfs

,特殊装备-耳环-悲剧人生的归寂,特殊装备-耳环-命运挑战者,特殊装备-耳环-次元流星坠,特殊装备-耳环-无尽地狱黑暗之印,特殊装备-耳环-电磁能量传送者,特殊装备-耳环-时之矛盾,特殊装备-耳环-宽容之海,特殊装备-耳环-军神的古怪耳环,特殊装备-耳环-无我灵晶,特殊装备-耳环-窥视未来耳环,...,未央幻境装备-玉荣-神器玉荣 [蓝色][红色]玉荣力130,未央幻境装备-玉荣-神器玉荣 [蓝色][红色]玉荣力100,未央幻境装备-玉荣-神器玉荣 [蓝色][绿色]玉荣力190,未央幻境装备-玉荣-神器玉荣 [蓝色][绿色]玉荣力160,未央幻境装备-玉荣-神器玉荣 [红色][绿色]玉荣力190,未央幻境装备-玉荣-神器玉荣 [蓝色][绿色]玉荣力130,未央幻境装备-玉荣-神器玉荣 [蓝色][绿色]玉荣力100,未央幻境装备-玉荣-神器玉荣 [红色][绿色]玉荣力160,未央幻境装备-玉荣-神器玉荣 [红色][绿色]玉荣力130,未央幻境装备-玉荣-神器玉荣 [红色][绿色]玉荣力100
日期,,,,,,,,,,,,,,,,,,,,,
17.06.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17.06.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17.06.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17.06.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17.06.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22.08.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4340418.0,2546515.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22.08.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4208641.0,2458676.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22.08.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3713579.0,2575342.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## output

In [20]:
version = 'v0.2.0-only-avg-price'
dfs.to_csv(f'sjm_dnf_price_{version}_output_{getCurTime()}.csv', encoding='utf_8_sig')

##  dev: locate bug based on unit of 100

In [216]:
## dfs.shape

def locateConcatBug(items, start, end):
    for pos in range(start+1, end):
        print(f'try concat {pos-1}-{pos}')
        dfs = pd.concat(map(item2df, items[pos-1:pos+1]), join='outer', ignore_index=False, axis=1,
        #                 keys=map(getItemName, items)
                       )

In [217]:
def locateConcatBug(items, start, end):
    for pos in range(start+1, end):
        print(f'try concat {pos-1}-{pos}')
        dfs = pd.concat(map(item2df, items[pos-1:pos+1]), join='outer', ignore_index=False, axis=1,
        #                 keys=map(getItemName, items)
                       )

In [184]:
locateConcatBug(itemsList, 700, 800)

try concat 700-701
try concat 701-702
try concat 702-703
try concat 703-704
try concat 704-705
try concat 705-706
try concat 706-707
try concat 707-708
try concat 708-709
try concat 709-710
try concat 710-711
try concat 711-712
try concat 712-713
try concat 713-714
try concat 714-715
try concat 715-716
try concat 716-717
try concat 717-718
try concat 718-719
try concat 719-720
try concat 720-721
try concat 721-722
try concat 722-723
try concat 723-724
try concat 724-725
try concat 725-726
try concat 726-727
try concat 727-728
try concat 728-729
try concat 729-730
try concat 730-731
try concat 731-732
try concat 732-733
try concat 733-734
try concat 734-735
try concat 735-736
try concat 736-737
try concat 737-738
try concat 738-739
try concat 739-740
try concat 740-741
try concat 741-742
try concat 742-743
try concat 743-744
try concat 744-745
try concat 745-746
try concat 746-747
try concat 747-748
try concat 748-749
try concat 749-750
try concat 750-751
try concat 751-752
try concat 7

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [181]:
SKIP = 600
LIMIT = 100

for skip in range(0, 1891, 100):
    print({"skip": skip, "limit": LIMIT})
    items = coll_price.find({}).skip(skip).limit(LIMIT)

    # todo: 700+ bug: InvalidIndexError: Reindexing only valid with uniquely valued Index objects
    dfs = pd.concat(map(item2df, items), join='outer', ignore_index=False, axis=1,
    #                 keys=map(getItemName, items)
                   )
    dfs

{'skip': 0, 'limit': 100}
{'skip': 100, 'limit': 100}
{'skip': 200, 'limit': 100}
{'skip': 300, 'limit': 100}
{'skip': 400, 'limit': 100}
{'skip': 500, 'limit': 100}
{'skip': 600, 'limit': 100}
{'skip': 700, 'limit': 100}


InvalidIndexError: Reindexing only valid with uniquely valued Index objects